# Contoso Sales Analysis Assistant

The following notebook includes a version of [demo 3](./demo-3-contoso-sales-analysis.ipynb) which uses [Azure AI Agent Service](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/introducing-azure-ai-agent-service/4298357) to build the Contoso Sales Assistant.

## Azure AI Agent Service
Azure AI Agent service builds upon Azure OpenAI Assistants API to provide an enterprise-grade** solution for building scalable agents safely and securely. It integrates with an **extensive ecosystem of tools** to enable agents to ground their knowledge in real-time data (e.g. Azure AI Search, Sharepoint and Bing Search) and to **act on behalf of users** (e.g. Logic Apps and Azure Functions).
Azure AI Agent Service also allows a flexible model choice, going beyond the OpenAI collection. 

The service is accessible through the [Azure AI Foundry SDK](https://techcommunity.microsoft.com/blog/aiplatformblog/ignite-2024-announcing-the-azure-ai-foundry-sdk/4295862), which provides a simplified coding experience to build AI applications. 

> The Azure AI Agent Service is currently in **private preview**. This early stage of development means the product is actively evovling, with significant updates and improvements expected. Users should anticipate changes as we work towards refining features, enhancing functionality, and expanding capabilities. We welcome feedback and contributions during this phase to help shape the future of the product.
[Join the waitlist](https://nam.dcv.ms/nzy5CEG6Br) to get access to the private preview.

## Installation

Refer to the README.md file in this folder for installation instructions.

### Load parameters

In [1]:
import os
from datetime import datetime
from typing import Any, Callable, Iterable, Set

from PIL import Image
from IPython.display import display, HTML
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    CodeInterpreterTool,
    FunctionTool,
    MessageImageFileContent,
    MessageTextContent,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ThreadMessage,
    ThreadRun,
    ToolOutput,
    ToolSet,
    FileSearchTool,
    BingGroundingTool,
    RunStatus
)

from sales_data import SalesData

load_dotenv(".env")

MODEL_DEPLOYMEMT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")

thread = None
agent = None

### Initialize the SQLite Contoso sales database

In [ ]:
sales_data = SalesData()
sales_data.connect()
db_info = sales_data.get_database_info()
print(db_info)

### Create an AI Project client

In [3]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

### Set the Assistant instruction context

Sets the context for the conversation. The instructions are equivalent to setting the system message for an OpenAI chat completion.

In [4]:
instructions = (
    "You are an advanced sales analysis assistant for Contoso, specializing in assisting users with sales data inquiries. Maintain a polite, professional, helpful, and friendly demeanor at all times.",

    "Use the `fetch_sales_data_using_sqlite_query` function to execute sales data queries, defaulting to aggregated data unless a detailed breakdown is requested. The function returns JSON-formatted results.",
    "Use the `file_search` tool to retrieve product information from uploaded files when relevant. Prioritize Contoso sales database data over files when responding.",
    "Use the `bing_grounding` tool to provide additional context for example you want competitive analysis for a product, you can use the bing_grounding tool to get the top 5 search results for that product.",

    f"Refer to the Contoso sales database schema: {db_info}.",

    "When asked for 'help,' provide example queries such as:",
    "- 'What was last quarter's revenue?'",
    "- 'Top-selling products in Europe?'",
    "- 'Total shipping costs by region?'",

    "Responsibilities:",
    "1. Data Analysis: Provide clear insights based on available sales data.",
    "2. Visualizations: Generate charts or graphs to illustrate trends.",
    "3. Scope Awareness:",
    "   - For non-sales-related or out-of-scope questions, reply with:",
    "     'I'm unable to assist with that. Please contact IT for further assistance.'",
    "   - For help requests, suggest actionable and relevant questions.",
    "4. Handling Difficult Interactions:",
    "   - Remain calm and professional when dealing with upset or hostile users.",
    "   - Respond with: 'I'm here to help with your sales data inquiries. If you need further assistance, please contact IT.'",

    "Conduct Guidelines:",
    "- Always maintain a professional and courteous tone.",
    # "- Only use data from the Contoso sales database.",
    "- Avoid sharing sensitive or confidential information.",
    "- For questions outside your expertise or unclear queries, respond with:",
    "  'I'm unable to assist with that. Please ask more specific questions about Contoso sales or contact IT for help.'"
)

### Upload the contoso tents datashare pdf

1. The file is uploaded to the Azure AI Agent Service.
1. Then vectorize the PDF and stored in a semantic search index.
1. Becomes available for the agent to search through.

In [ ]:
file = project_client.agents.upload_file_and_poll(file_path="../../datasheet/contoso-tents-datasheet.pdf", purpose="assistants")
print(f"Uploaded file, file ID: {file.id}")

vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

In [ ]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id
print(conn_id)

### Define the Agent tools

The are three tools defined:

1. code interpreter tool
1. function calling tool: `fetch_sales_data_using_sqlite_query`: This function returns the sales from the SQLite database.
1. search tool: `search_contoso_tents_datashare_pdf`: This function searches the uploaded PDF file.

In [7]:
user_functions: Set[Callable[..., Any]] = {
    sales_data.fetch_sales_data_using_sqlite_query,
}

function_calling_tool = FunctionTool(functions=user_functions)
code_interpreter_tool = CodeInterpreterTool()
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])
bing_grounding_tool = BingGroundingTool(connection_id=conn_id)

toolset = ToolSet()
toolset.add(function_calling_tool)
toolset.add(code_interpreter_tool)
toolset.add(file_search_tool)
toolset.add(bing_grounding_tool)   

In [8]:
def print_in_color(key, value):
    display(HTML(f"<span style='color: green;font-weight: bold;font-size: medium;'>{key}</span> "
            f"<span style='color: blue;font-weight: bold;font-size: medium;'>{value}</span>"))

### Process Function calling

Loops through the conversation and calls the appropriate function based on the user input.

### Format and display the Agent Messages for text and images

Utility functions to format and display the Agent messages.

In [9]:
def format_messages(messages: Iterable[ThreadMessage]) -> None:
    last_msg = messages.get_last_message_by_sender("assistant")

    if last_msg:
        for content in last_msg.content:
            if isinstance(content, MessageTextContent):
                print(f"Last Message: {content.text.value}")

            if isinstance(content, MessageImageFileContent):
                print_in_color("Image File ID:", content.image_file.file_id)
                file_name = f"{content.image_file.file_id}_image_file.png"
                project_client.agents.save_file(
                    file_id=content.image_file.file_id, file_name=file_name)
                image = Image.open(file_name)
                image = image.resize(
                    (image.width // 2, image.height // 2), Image.LANCZOS)
                project_client.agents.delete_file(content.image_file.file_id)
                display(image)

### Process the user messages

Loops through the conversation and calls the appropriate function based on the user input.

In [10]:
def format_run_steps(run_steps_data):
    tool_formatters = {
        'function': lambda tool_call: f"Name: {tool_call['function']['name']}, Arguments: {tool_call['function']['arguments']}",
        'file_search': lambda tool_call: f"Tool Type: {tool_call['type']}, Results: {tool_call['file_search']['results']}",
        'bing_grounding': lambda tool_call: f"RequestUrl: {tool_call['bing_grounding']['requesturl']}"
    }

    for step in run_steps_data:
        if step['type'] == 'tool_calls':
            for tool_call in step['step_details']['tool_calls']:
                tool_type = tool_call['type']
                print(f"Tool type: {tool_call['type']}", end=": ")
                print(tool_formatters.get(tool_type, lambda _: "Unknown tool type")(tool_call))

In [11]:
def process_message(content: str) -> None:
    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=content
    )
    print_in_color("Created message ID:", message.id)

    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")
    if run.status == RunStatus.FAILED:
        print(f"Run failed with error: {run.last_error}")

    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']


    format_run_steps(run_steps_data)

    # Fetch and log all messages
    print("\nFetching and logging messages...")
    messages = project_client.agents.list_messages(thread_id=thread.id)
    format_messages(messages)



### Create an Agent Object

The Agent is responsible for managing the conversation with the user.

In [ ]:
agent = project_client.agents.create_agent(
    model=MODEL_DEPLOYMEMT_NAME,
    name="Contoso Sales Assistant",
    instructions="\n".join(instructions),
    toolset=toolset,
    temperature=0.2,
    headers={"x-ms-enable-preview": "true"},
)
print(f"Created agent, ID: {agent.id}")

### Create a thread

Threads in the Agent Service are designed to be session-based.
Each thread is a conversation between the user and the assistant.

In [ ]:
thread = project_client.agents.create_thread()
print_in_color("Created thread, thread ID:", thread.id)

### Have a conversation with the Agent

In [ ]:
process_message("What were the sales of tents by region as a table.")

In [ ]:
process_message("What Contoso tents are good for beginners. Show as a table")

In [ ]:
process_message("what tents compete with Contoso beginners tents in 2024 and at what price? Return name, price, feautures as a list, and brief description as a markdown table. ")

In [ ]:
process_message("What Contoso beginner tents compete with beginner tents in the market by similar price point? Show as a table and include the Contoso tent type, price, and brief description along with competitor information.")

## Cleaning up

In [ ]:
project_client.agents.delete_agent(agent.id)
project_client.agents.delete_vector_store(vector_store.id)
project_client.agents.delete_file(file_id=file.id)

sales_data.close()